# [ Chapter 4 - Crowdsourced Relevance ] 
# Setting up the Retrotech Dataset

In [1]:
from aips import display_product_search, get_engine, set_engine
from aips.spark import get_spark_session
from aips.spark.dataframe import from_csv

set_engine("solr")
engine = get_engine()
print(type(engine))

<class 'engines.solr.SolrEngine.SolrEngine'>


In [2]:
spark = get_spark_session()

## Download the Retrotech (Ecommerce) Products + Signals Dataset

In [3]:
#Get datasets
![ ! -d 'retrotech' ] && git clone --depth 1 https://github.com/ai-powered-search/retrotech.git
! cd retrotech && git pull
! cd retrotech && mkdir -p '../data/retrotech/' && tar -xvf products.tgz -C '../data/retrotech/' && tar -xvf signals.tgz -C '../data/retrotech/'

Already up to date.
products.csv
signals.csv


## Get a Feel for the Product Catalog

### Listing 4.1

In [4]:
! cd data/retrotech/ && head products.csv

"upc","name","manufacturer","short_description","long_description"
"096009010836","Fists of Bruce Lee - Dolby - DVD", , , 
"043396061965","The Professional - Widescreen Uncut - DVD", , , 
"085391862024","Pokemon the Movie: 2000 - DVD", , , 
"067003016025","Summerbreeze - CD","Nettwerk", , 
"731454813822","Back for the First Time [PA] - CD","Def Jam South", , 
"024543008200","Big Momma's House - Widescreen - DVD", , , 
"031398751823","Kids - DVD", , , 
"037628413929","20 Grandes Exitos - CD","Sony Discos Inc.", , 
"060768972223","Power Of Trinity (Box) - CD","Sanctuary Records", , 


## Index the Products into the Search Engine

### Listing 4.2

In [10]:
from aips.data_loaders.products import load_dataframe

products_collection = engine.create_collection("products")
products_dataframe = load_dataframe("data/retrotech/products.csv")
products_collection.write(products_dataframe)

Wiping "products" collection
Creating "products" collection
Status: Success
Loading Products
Schema: 
root
 |-- upc: string (nullable = true)
 |-- name: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- long_description: string (nullable = true)
 |-- _text_: string (nullable = false)
 |-- name_ngram: string (nullable = false)
 |-- name_fuzzy: string (nullable = false)
 |-- short_description_ngram: string (nullable = false)
 |-- has_promotion: boolean (nullable = false)

Successfully written 48194 documents


## Verify Searches Work

### Listing 4.3

In [6]:
def product_search_request(query, param_overrides={}):
    request = {"query": query,
               "query_fields": ["name", "manufacturer", "long_description"],
               "return_fields": ["upc", "name", "manufacturer",
                                 "short_description", "score"],
               "limit": 5,
               "order_by": [("score", "desc"), ("upc", "asc")]}
    return request | param_overrides

In [7]:
query = "ipod"
request = product_search_request(query)
response = products_collection.search(**request)
display_product_search(query, response["docs"])

## Get a Feel for the Signals Data

In [8]:
! cd data/retrotech && head signals.csv

"query_id","user","type","target","signal_time"
"u2_0_1","u2","query","nook","2019-07-31 08:49:07.3116"
"u2_1_2","u2","query","rca","2020-05-04 08:28:21.1848"
"u3_0_1","u3","query","macbook","2019-12-22 00:07:07.0152"
"u4_0_1","u4","query","Tv antenna","2019-08-22 23:45:54.1030"
"u5_0_1","u5","query","AC power cord","2019-10-20 08:27:00.1600"
"u6_0_1","u6","query","Watch The Throne","2019-09-18 11:59:53.7470"
"u7_0_1","u7","query","Camcorder","2020-02-25 13:02:29.3089"
"u9_0_1","u9","query","wireless headphones","2020-04-26 04:26:09.7198"
"u10_0_1","u10","query","Xbox","2019-09-13 16:26:12.0132"


## Index the Signals into the Search Engine

### Listing 4.4

In [9]:
signals_collection = engine.create_collection("signals", log=True)
signals_collection.write(from_csv("data/retrotech/signals.csv"))

Wiping "signals" collection
Creating "signals" collection


{'responseHeader': {'status': 0, 'QTime': 829},
 'success': {'aips-solr:8983_solr': {'responseHeader': {'status': 0,
    'QTime': 268},
   'core': 'signals_shard1_replica_n1'}},
 'warning': 'Using _default configset. Data driven schema functionality is enabled by default, which is NOT RECOMMENDED for production use. To turn it off: curl http://{host:port}/solr/signals/config -d \'{"set-user-property": {"update.autoCreateFields":"false"}}\''}

Status: Success
Loading data/retrotech/signals.csv
Schema: 
root
 |-- query_id: string (nullable = true)
 |-- user: string (nullable = true)
 |-- type: string (nullable = true)
 |-- target: string (nullable = true)
 |-- signal_time: timestamp (nullable = true)

Successfully written 2172605 documents


## Success!

You have now indexed the RetroTech product catalog and signals into the search engine, and run a sample query against the product collection. The results don't look very relevant using the out of the box keyword scoring function, of course, but we'll be working to improve that throughout the rest of this book!

In the next section, we'll take a look at our first crowd-sourced AI-powered search technique: [Signals Boosting](2.signals-boosting.ipynb). 